<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/nptebooks/5_1_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация CatBoostClassifier с использованием GPU (CUDA 12.5) для классификации узлов на датасете Bitcoin-OTC

Задача — предсказать "уровень доверия" пользователей

Признаки узлов/рёбер - синтетические

Небходимо выбрать среду выполнения с GPU: Среда выполнения → Сменить среду выполнения → Графический процессор T4

## 1. Установка зависимостей

Требования:
- CatBoost ≥1.2.2
- PyTorch Geometric для загрузки данных


* CatBoost автоматически использует доступные GPU через CUDA, явная установка CUDA 12.5 не требуется

In [1]:
install = True
if install:
  !pip install -q catboost
  !pip install -q torch-geometric

## 2. Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from torch_geometric.datasets import BitcoinOTC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 3. Загрузка данных

In [3]:
dataset = BitcoinOTC(root='/tmp/BitcoinOTC')
data = dataset[0]

## 4. Создание признаков узлов с учётом рёбер

In [4]:
def create_node_features(data):
    num_nodes = data.num_nodes
    features = []

    # Степень узлов
    degrees = np.zeros(num_nodes)
    for src in data.edge_index[0].unique().cpu().numpy():
        degrees[src] = (data.edge_index[0].cpu().numpy() == src).sum()

    # Статистики по рёбрам
    edge_attrs = [[] for _ in range(num_nodes)]
    for i in range(data.edge_index.size(1)):
        src = data.edge_index[0, i].item()
        attr = data.edge_attr[i].item()
        edge_attrs[src].append(attr)

    # Создание фичей
    for node_id in range(num_nodes):
        node_feats = [
            degrees[node_id],  # Степень узла
            np.mean(edge_attrs[node_id]) if edge_attrs[node_id] else 0,  # Среднее значение рёбер
            np.max(edge_attrs[node_id]) if edge_attrs[node_id] else 0,   # Максимум
            np.min(edge_attrs[node_id]) if edge_attrs[node_id] else 0,   # Минимум
            len(edge_attrs[node_id])                                      # Количество рёбер
        ]
        features.append(node_feats)

    return np.array(features)

## 5. Подготовка данных

In [5]:
X = create_node_features(data)
X

array([[2. , 2.5, 4. , 1. , 2. ],
       [3. , 5. , 5. , 5. , 3. ],
       [0. , 0. , 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]])

In [6]:
y = np.zeros(data.num_nodes)

# Создание меток (3 класса)
q1 = np.quantile(X[:, 0], 0.5)
q2 = np.quantile(X[:, 0], 0.999)
y[X[:, 0] > q2] = 2
y[(X[:, 0] > q1) & (X[:, 0] <= q2)] = 1

## 6. Разделение данных

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

## 7. Создание CatBoost модели с GPU

In [8]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    task_type='GPU',  # Включение GPU
    devices='0:1',    # Использование первого GPU
    verbose=100,
    early_stopping_rounds=50
)

## 8. Обучение

In [9]:
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9095124	test: 0.9095340	best: 0.9095340 (0)	total: 60.5ms	remaining: 1m
100:	learn: 0.0003896	test: 0.0004030	best: 0.0004030 (100)	total: 1.29s	remaining: 11.5s
200:	learn: 0.0001095	test: 0.0001153	best: 0.0001153 (200)	total: 5.19s	remaining: 20.6s
300:	learn: 0.0000632	test: 0.0000670	best: 0.0000670 (300)	total: 10.4s	remaining: 24.2s
400:	learn: 0.0000442	test: 0.0000470	best: 0.0000470 (400)	total: 14.5s	remaining: 21.7s
500:	learn: 0.0000339	test: 0.0000360	best: 0.0000360 (500)	total: 18.9s	remaining: 18.8s
600:	learn: 0.0000275	test: 0.0000293	best: 0.0000293 (600)	total: 20.9s	remaining: 13.9s
700:	learn: 0.0000231	test: 0.0000246	best: 0.0000246 (700)	total: 21.8s	remaining: 9.28s
800:	learn: 0.0000199	test: 0.0000211	best: 0.0000211 (800)	total: 22.5s	remaining: 5.59s
900:	learn: 0.0000174	test: 0.0000185	best: 0.0000185 (900)	total: 23.2s	remaining: 2.55s
999:	learn: 0.0000157	test: 0.0000167	best: 0.0000167 (999)	total: 24s	remaining: 0us
bestTest = 1.6675111

## 9. Оценка

In [10]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1198
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1

    accuracy                           1.00      1201
   macro avg       1.00      1.00      1.00      1201
weighted avg       1.00      1.00      1.00      1201



## 10. Важность признаков

In [11]:
feature_names = ['Degree', 'EdgeMean', 'EdgeMax', 'EdgeMin', 'EdgeCount']
print("\nFeature Importance:")
for name, score in zip(feature_names, model.get_feature_importance()):
    print(f"{name}: {score:.2f}")


Feature Importance:
Degree: 65.54
EdgeMean: 2.91
EdgeMax: 6.74
EdgeMin: 0.02
EdgeCount: 24.79


## 11. Сохранение модели

In [12]:
model.save_model('catboost_model.cbm')

## 12. Загрузка модели

In [13]:
loaded_model = CatBoostClassifier()
loaded_model.load_model('catboost_model.cbm')

In [14]:
y_pred = loaded_model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1198
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1

    accuracy                           1.00      1201
   macro avg       1.00      1.00      1.00      1201
weighted avg       1.00      1.00      1.00      1201

